## **Entrega Final: Automatización de un Pipeline de Machine Learning con GitHub Actions**  
### Johanna Peña

In [14]:
#Cargue de datos
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets   
# metadata 
print(iris.metadata) 
  
# variable information 
print(iris.variables) 

{'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'ID': 191, 'type': 'NATIVE', 'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'venue': 'Significance, 2021', 'year': 2021, 'journal': 'Significance, 2021', 'DOI': '1740-9713.01589', 'URL': 'https://www.semanticscholar.org

In [15]:
# Preprocesamiento y Entrenamiento del Modelo
# Importar bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn

# Convertir los datos en un DataFrame para inspección
df = pd.DataFrame(X, columns=iris.feature_names)

# Revisión de valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# Revisar duplicados
print("\nNúmero de duplicados:", df.duplicated().sum())

Valores nulos por columna:
sepal length    0
sepal width     0
petal length    0
petal width     0
dtype: int64

Número de duplicados: 3


In [16]:
# Eliminar duplicados
df = df.drop_duplicates()
# Revisar duplicados
print("\nNúmero de duplicados:", df.duplicated().sum())


Número de duplicados: 0


In [17]:
# Revisar distribución de las clases
# Asegurarse de que y sea un vector unidimensional
y = y.values.ravel()  # Convierte a unidimensional si no lo es

# Ahora podemos usar pd.Series para verificar la distribución de las clases
print("\nDistribución de las clases (target):")
print(pd.Series(y).value_counts())

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Asegurarse de que y esté en la forma correcta (unidimensional)
y_train = y_train.ravel()  # Aplana el array a una dimensión
y_test = y_test.ravel()

# Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Hacer predicciones y evaluar
y_pred = model.predict(X_test_scaled)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy}")

# Imprimir reporte de clasificación (precisión, recall, F1-score)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Imprimir matriz de confusión
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

# Iniciar el seguimiento con MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.start_run()

# Registrar parámetros del modelo
mlflow.log_param("n_estimators", 100)

# Registrar métricas del modelo
mlflow.log_metric("accuracy", accuracy)

# Registrar reporte de clasificación como texto
mlflow.log_text(str(classification_report(y_test, y_pred)), "classification_report.txt")

# Crear un ejemplo de entrada
input_example = X_test_scaled[0].reshape(1, -1)

# Registrar el modelo con ejemplo de entrada
mlflow.sklearn.log_model(model, "model", input_example=input_example)

# Finalizar el seguimiento
mlflow.end_run()


Distribución de las clases (target):
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

Accuracy: 1.0

Reporte de clasificación:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30


Matriz de Confusión:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
